In [1]:
# to load dataset
from datasets import Datasets

# for network and training
from network import Net_teacher

# to calculate the score
import savescore
from score import Score
from score_calc import ScoreCalc

# pytorch
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# initialize for  each parameters
DATASET = 'CIFAR10'
BATCH_SIZE = 100
NUM_WORKERS = 2

In [3]:
# check if gpu is available
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("gpu mode")
else:
    device = torch.device("cpu")
    print("cpu mode")

gpu mode


In [4]:
# the name of results files
codename = 'kd_teacher'

result_name = codename + "_result"

In [5]:
# load the data set
instance_datasets = Datasets(DATASET, BATCH_SIZE, NUM_WORKERS)
data_sets = instance_datasets.create()

trainloader = data_sets[0]
testloader = data_sets[1]
classes = data_sets[2]
based_labels = data_sets[3]
trainset = data_sets[4]
testset = data_sets[5]

Dataset : CIFAR10
Files already downloaded and verified
Files already downloaded and verified


In [6]:
# network and criterions
model_t = Net_teacher().to(device)
model_t.load_state_dict(torch.load("cnn_alex.pkl"))
model_t.eval()

soft_criterion = nn.CrossEntropyLoss()

In [7]:
# to manage all scores
loss = Score()
correct = Score()
score_loss = [loss]
score_correct = [correct]
sc = ScoreCalc(score_loss, score_correct, BATCH_SIZE)

In [8]:
for (inputs, labels) in trainloader:
    inputs, labels = inputs.to(device), labels.to(device)

    out = model_t(inputs)
    
    loss = soft_criterion(out, labels).item()
    
    _, predicted = out.max(1)
    correct = (predicted == labels).sum().item()
    
    sc.calc_sum([loss], [correct])

sc.score_print(len(trainset))
sc.score_append(len(trainset))
sc.score_del()

for (inputs, labels) in testloader:
    inputs, labels = inputs.to(device), labels.to(device)

    out = model_t(inputs)
    
    loss = soft_criterion(out, labels).item()
    
    _, predicted = out.max(1)
    correct = (predicted == labels).sum().item()
    
    sc.calc_sum([loss], [correct])

sc.score_print(len(testset), train = False)
sc.score_append(len(testset), train = False)
sc.score_del()

train mean loss=0.18938172137737275, accuracy=0.99196
test mean loss=0.5745016613602638, accuracy=0.8375


In [9]:
# get the scores
train_losses, train_corrects = sc.get_value()
test_losses, test_corrects = sc.get_value(train = False)

In [12]:
# output the scores
savescore.save_data(train_losses[0], test_losses[0], train_corrects[0], test_corrects[0], result_name)